### Components

In [1]:
from lib.components.wiki_hierarchy_builder import WikiHierarchyBuilder
from lib.components.wiki_context_creator import WikiContextCreator
from lib.templates import hierarchy_template, final_template
from haystack.components.builders import PromptBuilder
from haystack.components.generators import OpenAIGenerator
from haystack_integrations.document_stores.elasticsearch import ElasticsearchDocumentStore

wiki_hierarchy_builder = WikiHierarchyBuilder("bolt://localhost:7687", "neo4j", "neo4jpass")
hierarchy_prompt_builder = PromptBuilder(template=hierarchy_template)
hierarchy_generator = OpenAIGenerator(model="gpt-4o-mini")
elasticsearch_store = ElasticsearchDocumentStore(hosts= "http://localhost:9200")
wiki_context_creator = WikiContextCreator(document_store=elasticsearch_store)
final_prompt_builder = PromptBuilder(template=final_template)
final_generator = OpenAIGenerator(model="gpt-4o-mini")


/home/kartikeya/miniconda3/envs/argonk/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Pipeline

In [2]:
from haystack import Pipeline

graph_pipeline = Pipeline()

graph_pipeline.add_component("wiki_hierarchy_builder", wiki_hierarchy_builder)
graph_pipeline.add_component("hierarchy_prompt_builder", hierarchy_prompt_builder)
graph_pipeline.add_component("hierarchy_generator", hierarchy_generator)
graph_pipeline.add_component("wiki_context_creator", wiki_context_creator)
graph_pipeline.add_component("final_prompt_builder", final_prompt_builder)
graph_pipeline.add_component("final_generator", final_generator)

graph_pipeline.connect("wiki_hierarchy_builder.sections_hierarchy", "hierarchy_prompt_builder.hierarchy")
graph_pipeline.connect("hierarchy_prompt_builder", "hierarchy_generator")
graph_pipeline.connect("hierarchy_generator.replies", "wiki_context_creator.hierarchy_paths")
graph_pipeline.connect("wiki_hierarchy_builder.chunks_hierarchy", "wiki_context_creator.chunks_hierarchy")
graph_pipeline.connect("wiki_context_creator.context_list", "final_prompt_builder.context_list")
graph_pipeline.connect("final_prompt_builder", "final_generator")

🚅 Components
  - wiki_hierarchy_builder: WikiHierarchyBuilder
  - hierarchy_prompt_builder: PromptBuilder
  - hierarchy_generator: OpenAIGenerator
  - wiki_context_creator: WikiContextCreator
  - final_prompt_builder: PromptBuilder
  - final_generator: OpenAIGenerator
🛤️ Connections
  - wiki_hierarchy_builder.sections_hierarchy -> hierarchy_prompt_builder.hierarchy (dict)
  - wiki_hierarchy_builder.chunks_hierarchy -> wiki_context_creator.chunks_hierarchy (dict)
  - hierarchy_prompt_builder.prompt -> hierarchy_generator.prompt (str)
  - hierarchy_generator.replies -> wiki_context_creator.hierarchy_paths (List[str])
  - wiki_context_creator.context_list -> final_prompt_builder.context_list (List[str])
  - final_prompt_builder.prompt -> final_generator.prompt (str)

In [3]:
graph_pipeline.draw("graph_pipeline.png")

### Pipeline input data

In [3]:
from haystack import Document
from lib.models.hierarchy_path import HierarchyPathData

docs = [
    Document(
        id="3f45e988-b093-4d9c-bde1-67e2e4c3b599",
        content="All non-avian dinosaurs and most lineages of birds became extinct in a mass extinction event, called...",
        meta={
            "file_path": "Dinosaur.html",
            "source_id": "93000a3fb02b99d2d115cd4042256d2f5db2a0ff3928927ca14465276534a75e",
            "split_id": 234,
            "title": "Dinosaurs",
            "h2": "Extinction of major groups",
        },
    ),
    Document(
        id="433ad671-ea22-4ae1-9bc4-1cce06a0e6ee",
        content="Just before the K-Pg extinction event, the number of non-avian dinosaur species that existed globall...",
        meta={
            "file_path": "Dinosaur.html",
            "source_id": "93000a3fb02b99d2d115cd4042256d2f5db2a0ff3928927ca14465276534a75e",
            "split_id": 235,
            "title": "Dinosaurs",
            "h2": "Extinction of major groups",
            "h3": "Pre-extinction diversity",
        },
    ),
]

query = "Give a brief about alternative theories of dinosaur extinction."

path_schema = HierarchyPathData.model_json_schema()



In [4]:
input_data = {
    "hierarchy_prompt_builder": {"query": query, "hierarchy_path_schema": path_schema},
    "wiki_hierarchy_builder": {"documents": docs},
    "final_prompt_builder": {"query": query},
}

### Run pipeline

In [5]:
result = graph_pipeline.run(
    data=input_data,
    include_outputs_from={
        "hierarchy_generator",
        "wiki_context_creator",
        "final_generator",
    },
)

In [6]:
result['hierarchy_generator']

{'meta': [{'model': 'gpt-4o-mini-2024-07-18',
   'index': 0,
   'finish_reason': 'stop',
   'usage': {'completion_tokens': 56,
    'prompt_tokens': 1280,
    'total_tokens': 1336,
    'completion_tokens_details': {'reasoning_tokens': 0}}}],
 'replies': ['{\n  "paths": [\n    {\n      "path": ["Dinosaur", "Extinction of major groups"],\n      "reasoning": "The \'Extinction of major groups\' section is likely to address various theories related to the extinction of dinosaurs."\n    }\n  ]\n}']}

In [7]:
result['wiki_context_creator']

{'context_list': ['Extinction of major groups (h2):\nAll non-avian dinosaurs and most lineages of birds became extinct in a mass extinction event, called the Cretaceous–Paleogene (K-Pg) extinction event, at the end of the Cretaceous period. Above the Cretaceous–Paleogene boundary, which has been dated to 66.038 ± 0.025 million years ago, fossils of non-avian dinosaurs disappear abruptly; the absence of dinosaur fossils was historically used to assign rocks to the ensuing Cenozoic. The nature of the event that caused this mass extinction has been extensively studied since the 1970s, leading to the development of two mechanisms that are thought to have played major roles: an extraterrestrial impact event in the Yucatán Peninsula, along with flood basalt volcanism in India. However, the specific mechanisms of the extinction event and the extent of its effects on dinosaurs are still areas of ongoing research. Alongside dinosaurs, many other groups of animals became extinct: pterosaurs, mar

In [8]:
result['final_generator']

{'replies': ['The context provides information on two major mechanisms related to the K-Pg extinction event: the extraterrestrial impact event at the Yucatán Peninsula and flood basalt volcanism in India (Deccan Traps). \n\n1. **Extraterrestrial Impact**: The bolide impact hypothesis suggests that a significant impact caused immediate effects like earthquakes, tsunamis, and a global firestorm, leading to the extinction event. The impact also had long-term consequences such as cooling due to soot aerosols and collapse of food webs.\n\n2. **Deccan Traps**: The Deccan Traps flood basalts were actively erupting around the K-Pg boundary, releasing greenhouse gases and causing climate change. The role of these eruptions in the extinction remains unclear, with debates on whether they were a major cause of the extinction, alongside the Chicxulub impact. \n\nThere is uncertainty about how these two mechanisms interacted, with some suggesting that the impact from Chicxulub may have worsened the 

In [10]:
print(result['final_generator']['replies'][0])

The context provides information on two major mechanisms related to the K-Pg extinction event: the extraterrestrial impact event at the Yucatán Peninsula and flood basalt volcanism in India (Deccan Traps). 

1. **Extraterrestrial Impact**: The bolide impact hypothesis suggests that a significant impact caused immediate effects like earthquakes, tsunamis, and a global firestorm, leading to the extinction event. The impact also had long-term consequences such as cooling due to soot aerosols and collapse of food webs.

2. **Deccan Traps**: The Deccan Traps flood basalts were actively erupting around the K-Pg boundary, releasing greenhouse gases and causing climate change. The role of these eruptions in the extinction remains unclear, with debates on whether they were a major cause of the extinction, alongside the Chicxulub impact. 

There is uncertainty about how these two mechanisms interacted, with some suggesting that the impact from Chicxulub may have worsened the climate changes init